<a href="https://colab.research.google.com/github/raycmarange/AML425/blob/main/AutoencoderAssng1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Autoencoder Assignment 1
#name: [ray marange]
#date: [11/09/2025]
#email: [rayc.marange@gmail.com]
# Description: Implementation of an autoencoder with MSE objective, latent space control,
#              and generative capabilities, along with analysis of information-theoretic properties.
# Note: This code is structured to follow the assignment requirements step-by-step.
#university of VICTORIA WELLINGTON NEW ZEALAND
import numpy as np
import tensorflow as tf
layers = tf.keras.layers
models = tf.keras.models
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy, multivariate_normal
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

###############################################################################
# 1. Create 3D data uniformly distributed over the surface of a cube
###############################################################################
print("=== 1. Generating 3D Cube Surface Data ===")

def generate_cube_surface_data(n_samples=10000):
    """
    Generate points uniformly distributed on the surface of a cube centered at origin with side length 2
    Returns:
        numpy array of shape (n_samples, 3) containing the 3D points
    """
    samples_per_face = n_samples // 6

    data = []
    # For each face of the cube (x=±1, y=±1, z=±1)
    for fixed_coord in [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0, -1), (0, 0, 1)]:
        if len(fixed_coord) == 2:  # x or y is fixed
            axis, value = fixed_coord
            if axis == -1:  # x = -1
                x = np.full(samples_per_face, -1)
                y = np.random.uniform(-1, 1, samples_per_face)
                z = np.random.uniform(-1, 1, samples_per_face)
            elif axis == 1:  # x = 1
                x = np.full(samples_per_face, 1)
                y = np.random.uniform(-1, 1, samples_per_face)
                z = np.random.uniform(-1, 1, samples_per_face)
            elif axis == 0 and value == -1:  # y = -1
                x = np.random.uniform(-1, 1, samples_per_face)
                y = np.full(samples_per_face, -1)
                z = np.random.uniform(-1, 1, samples_per_face)
            else:  # y = 1
                x = np.random.uniform(-1, 1, samples_per_face)
                y = np.full(samples_per_face, 1)
                z = np.random.uniform(-1, 1, samples_per_face)
        else:  # z is fixed
            _, _, value = fixed_coord
            if value == -1:  # z = -1
                x = np.random.uniform(-1, 1, samples_per_face)
                y = np.random.uniform(-1, 1, samples_per_face)
                z = np.full(samples_per_face, -1)
            else:  # z = 1
                x = np.random.uniform(-1, 1, samples_per_face)
                y = np.random.uniform(-1, 1, samples_per_face)
                z = np.full(samples_per_face, 1)

        face_data = np.column_stack((x, y, z))
        data.append(face_data)

    data = np.vstack(data)
    np.random.shuffle(data)
    return data

# Generate the data
cube_data = generate_cube_surface_data(6000)
cube_data = cube_data.astype(np.float32)
print(f"Generated {cube_data.shape[0]} samples on cube surface")
print(f"Data range: x[{np.min(cube_data[:, 0]):.2f}, {np.max(cube_data[:, 0]):.2f}], "
      f"y[{np.min(cube_data[:, 1]):.2f}, {np.max(cube_data[:, 1]):.2f}], "
      f"z[{np.min(cube_data[:, 2]):.2f}, {np.max(cube_data[:, 2]):.2f}]")

# Visualize the cube surface data
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
sample_idx = np.random.choice(len(cube_data), 1000, replace=False)
ax.scatter(cube_data[sample_idx, 0], cube_data[sample_idx, 1], cube_data[sample_idx, 2], alpha=0.5)
ax.set_title('Generated 3D Cube Surface Data')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1.5, 1.5)
plt.savefig('cube_surface_data.png')
plt.close()

# Split into train and test
train_data, test_data = train_test_split(cube_data, test_size=0.2, random_state=42)

###############################################################################
# 2. Autoencoder with MSE objective and adjustable latent size
#    Enhanced with MMD regularization for latent space
###############################################################################
print("\n=== 2. Building Autoencoder with MSE Objective and MMD Regularization ===")

def compute_mmd(x, y, sigma=1.0):
    """
    Compute Maximum Mean Discrepancy (MMD) between two samples
    Uses Gaussian kernel with specified sigma
    """
    x = tf.convert_to_tensor(x, dtype=tf.float32)  # Ensure float32
    y = tf.convert_to_tensor(y, dtype=tf.float32)  # Ensure float32

    xx = tf.reduce_mean(tf.exp(-tf.reduce_sum(tf.square(x[:, None] - x[None, :]), axis=-1) / (2 * sigma**2)))
    yy = tf.reduce_mean(tf.exp(-tf.reduce_sum(tf.square(y[:, None] - y[None, :]), axis=-1) / (2 * sigma**2)))
    xy = tf.reduce_mean(tf.exp(-tf.reduce_sum(tf.square(x[:, None] - y[None, :]), axis=-1) / (2 * sigma**2)))

    return xx + yy - 2 * xy

def build_autoencoder(latent_dim=2, mmd_weight=0.1):
    """
    Build an autoencoder with adjustable latent dimension and MMD regularization
    Args:
        latent_dim: Dimension of the latent space
        mmd_weight: Weight for the MMD regularization term
    Returns:
        autoencoder, encoder, decoder models
    """
    # Encoder
    encoder_input = layers.Input(shape=(3,))
    x = layers.Dense(64, activation='relu')(encoder_input)
    x = layers.Dense(32, activation='relu')(x)
    latent = layers.Dense(latent_dim, activation='linear', name='latent')(x)
    encoder_model = models.Model(encoder_input, latent)

    # Decoder
    latent_input = layers.Input(shape=(latent_dim,))
    x = layers.Dense(32, activation='relu')(latent_input)
    x = layers.Dense(64, activation='relu')(x)
    decoder_output = layers.Dense(3, activation='linear')(x)
    decoder_model = models.Model(latent_input, decoder_output)

    # Autoencoder
    autoencoder_input = layers.Input(shape=(3,))
    encoded = encoder_model(autoencoder_input)
    decoded = decoder_model(encoded)
    autoencoder_model = models.Model(autoencoder_input, decoded)

    # Custom training with MMD regularization
    optimizer = tf.keras.optimizers.Adam()
    loss_metric = tf.keras.metrics.Mean(name='loss')
    recon_loss_metric = tf.keras.metrics.Mean(name='reconstruction_loss')
    mmd_loss_metric = tf.keras.metrics.Mean(name='mmd_loss')

    @tf.function
    def train_step(data):
        with tf.GradientTape() as tape:
            # Forward pass
            latent = encoder_model(data, training=True)
            reconstructed = decoder_model(latent, training=True)

            # Reconstruction loss
            reconstruction_loss = tf.reduce_mean(tf.square(data - reconstructed))

            # MMD regularization - match latent distribution to standard normal
            true_samples = tf.random.normal(tf.shape(latent))
            mmd_loss = compute_mmd(latent, true_samples)

            # Total loss
            total_loss = reconstruction_loss + mmd_weight * mmd_loss

        # Compute gradients
        grads = tape.gradient(total_loss,
                             encoder_model.trainable_variables + decoder_model.trainable_variables)
        optimizer.apply_gradients(
            zip(grads, encoder_model.trainable_variables + decoder_model.trainable_variables))

        # Update metrics
        loss_metric.update_state(total_loss)
        recon_loss_metric.update_state(reconstruction_loss)
        mmd_loss_metric.update_state(mmd_loss)

        return {
            "loss": loss_metric.result(),
            "reconstruction_loss": recon_loss_metric.result(),
            "mmd_loss": mmd_loss_metric.result(),
        }

    # Create a custom model class to handle training
    class CustomAutoencoder(models.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super(CustomAutoencoder, self).__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder

        def call(self, inputs):
            latent = self.encoder(inputs)
            return self.decoder(latent)

        def train_step(self, data):
            return train_step(data)

        @property
        def metrics(self):
            return [loss_metric, recon_loss_metric, mmd_loss_metric]

    # Create and compile the model
    autoencoder = CustomAutoencoder(encoder_model, decoder_model)
    autoencoder.compile(optimizer=optimizer)

    return autoencoder, encoder_model, decoder_model

# Build and train autoencoder with different latent dimensions
latent_dims = [1, 2, 4, 8, 16]
autoencoders = {}
histories = {}

for dim in latent_dims:
    print(f"Training autoencoder with latent dimension {dim}")
    autoencoder, encoder, decoder = build_autoencoder(latent_dim=dim, mmd_weight=0.1)

    # Custom training loop
    epochs = 100
    batch_size = 32
    n_batches = int(np.ceil(len(train_data) / batch_size))

    history = {'loss': [], 'reconstruction_loss': [], 'mmd_loss': [], 'val_loss': []}

    for epoch in range(epochs):
        # Shuffle training data
        indices = np.random.permutation(len(train_data))
        train_data_shuffled = train_data[indices]

        # Reset metrics
        for metric in autoencoder.metrics:
            metric.reset_state()  # Fixed: reset_state() instead of reset_states()

        # Train on batches
        for batch_idx in range(n_batches):
            batch_start = batch_idx * batch_size
            batch_end = min((batch_idx + 1) * batch_size, len(train_data))
            batch_data = train_data_shuffled[batch_start:batch_end]

            # Train step
            autoencoder.train_step(batch_data)

        # Record training metrics
        history['loss'].append(autoencoder.metrics[0].result().numpy())
        history['reconstruction_loss'].append(autoencoder.metrics[1].result().numpy())
        history['mmd_loss'].append(autoencoder.metrics[2].result().numpy())

        # Calculate validation loss
        val_reconstructions = autoencoder.predict(test_data, verbose=0)
        val_loss = np.mean(np.square(test_data - val_reconstructions))
        history['val_loss'].append(val_loss)

        if epoch % 20 == 0:
            print(f"Epoch {epoch}: loss={history['loss'][-1]:.4f}, val_loss={val_loss:.4f}")

    autoencoders[dim] = (autoencoder, encoder, decoder)
    histories[dim] = history

    # Plot training history
    plt.figure(figsize=(8, 4))
    plt.plot(history['loss'], label='Total Loss')
    plt.plot(history['reconstruction_loss'], label='Reconstruction Loss')
    plt.plot(history['mmd_loss'], label='MMD Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Training History - Latent Dim: {dim}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'training_history_dim_{dim}.png')
    plt.close()

###############################################################################
# 3. Probabilistic interpretation of MSE objective function
###############################################################################
print("\n=== 3. Probabilistic Interpretation of MSE ===")
print("The MSE objective function can be interpreted as maximizing the log-likelihood")
print("of the data under a Gaussian distribution with fixed variance.")
print("Specifically, minimizing MSE is equivalent to maximizing:")
print("log p(X|Z) = -1/(2σ²) * MSE(X, X̂) + constant")
print("where σ² is the fixed variance of the Gaussian noise model.")
print("This assumes that the reconstruction errors are normally distributed with mean 0.")
print("This corresponds to Maximum Likelihood Estimation (MLE) under the Gaussian assumption.")

###############################################################################
# 4. Method to control latent distribution and add noise
###############################################################################
print("\n=== 4. Controlling Latent Distribution with Noise ===")

def add_latent_noise(encoder, data, snr_db):
    """
    Add Gaussian noise to latent representation with specified SNR
    Returns noisy latent codes and the information rate
    Args:
        encoder: Encoder model
        data: Input data
        snr_db: Signal-to-noise ratio in dB
    Returns:
        noisy_latent: Latent codes with added noise
        total_information: Information rate in bits/sample
        signal_power: Power of the signal
        noise_power: Power of the noise
    """
    # Get latent representation
    latent_codes = encoder.predict(data, verbose=0)

    # Calculate signal power (variance)
    signal_power = np.var(latent_codes, axis=0)

    # Convert SNR from dB to linear scale
    snr_linear = 10**(snr_db / 10)

    # Calculate noise power for each dimension
    noise_power = signal_power / snr_linear

    # Add Gaussian noise
    noise = np.random.normal(0, np.sqrt(noise_power), latent_codes.shape)
    noisy_latent = latent_codes + noise

    # Calculate information rate using equation (2)
    # I(Y;Z) = 0.5 * log2(σ_z² / σ_ε²) = 0.5 * log2(SNR) for each dimension
    information_rate = 0.5 * np.log2(1 + snr_linear)  # bits per dimension per sample

    total_information = np.sum(information_rate)

    return noisy_latent, total_information, signal_power, noise_power

# Test with different SNRs
snr_values = [-10, -5, 0, 5, 10, 20, 30]  # in dB
latent_dim = 2
encoder = autoencoders[latent_dim][1]

print("\nInformation Rate at Different SNRs:")
print("SNR (dB)\tInfo Rate (bits/sample)\tSignal Power\tNoise Power")
for snr_db in snr_values:
    _, info_rate, signal_power, noise_power = add_latent_noise(encoder, test_data, snr_db)
    print(f"{snr_db}\t\t{info_rate:.4f}\t\t\t{np.mean(signal_power):.4f}\t\t{np.mean(noise_power):.4f}")

###############################################################################
# 5. Analyze reconstructions at various SNRs and latent dimensions
###############################################################################
print("\n=== 5. Analyzing Reconstructions at Various SNRs and Latent Dimensions ===")

def evaluate_reconstruction(encoder, decoder, data, snr_db):
    """
    Evaluate reconstruction quality with added noise
    Args:
        encoder: Encoder model
        decoder: Decoder model
        data: Input data
        snr_db: Signal-to-noise ratio in dB
    Returns:
        mse: Mean squared error of reconstruction
        info_rate: Information rate in bits/sample
        reconstructed: Reconstructed data
    """
    # Get noisy latent codes
    noisy_latent, info_rate, _, _ = add_latent_noise(encoder, data, snr_db)

    # Reconstruct
    reconstructed = decoder.predict(noisy_latent, verbose=0)

    # Calculate reconstruction error
    mse = np.mean(np.square(data - reconstructed))

    return mse, info_rate, reconstructed

# Evaluate for different latent dimensions and SNRs
results = {}

print("\nReconstruction Quality Analysis:")
print("Dim\tSNR (dB)\tMSE\t\tInfo Rate (bits/sample)")
for dim in latent_dims:
    encoder = autoencoders[dim][1]
    decoder = autoencoders[dim][2]

    results[dim] = {}

    for snr_db in snr_values:
        mse, info_rate, reconstructed = evaluate_reconstruction(encoder, decoder, test_data, snr_db)
        results[dim][snr_db] = {'mse': mse, 'info_rate': info_rate}
        print(f"{dim}\t{snr_db}\t\t{mse:.6f}\t{info_rate:.4f}")

# Visualize reconstructions for a specific latent dimension and SNR
dim = 2
snr_db = 10
encoder = autoencoders[dim][1]
decoder = autoencoders[dim][2]

_, _, reconstructed = evaluate_reconstruction(encoder, decoder, test_data, snr_db)

# Plot original and reconstructed
fig = plt.figure(figsize=(15, 5))

# Original data
ax1 = fig.add_subplot(131, projection='3d')
sample_idx = np.random.choice(len(test_data), 500, replace=False)
ax1.scatter(test_data[sample_idx, 0], test_data[sample_idx, 1], test_data[sample_idx, 2], alpha=0.5)
ax1.set_title('Original Test Data')
ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_zlim(-1.5, 1.5)

# Reconstructed data
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(reconstructed[sample_idx, 0], reconstructed[sample_idx, 1], reconstructed[sample_idx, 2], alpha=0.5)
ax2.set_title(f'Reconstructed (Dim={dim}, SNR={snr_db}dB)')
ax2.set_xlim(-1.5, 1.5)
ax2.set_ylim(-1.5, 1.5)
ax2.set_zlim(-1.5, 1.5)

# Error visualization
errors = np.linalg.norm(test_data - reconstructed, axis=1)
ax3 = fig.add_subplot(133)
ax3.hist(errors, bins=50)
ax3.set_title('Reconstruction Error Distribution')
ax3.set_xlabel('Euclidean Distance Error')
ax3.set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('reconstruction_comparison.png')
plt.close()

###############################################################################
# 6. Create a generative system and define quality measures
###############################################################################
print("\n=== 6. Creating Generative System with Quality Measures ===")

def create_generative_system(encoder, decoder, data):
    """
    Create a generative system from the autoencoder
    Args:
        encoder: Encoder model
        decoder: Decoder model
        data: Training data to fit the latent distribution
    Returns:
        generate_samples: Function to generate new samples
        latent_mean: Mean of the latent distribution
        latent_cov: Covariance of the latent distribution
    """
    # Get latent representations of training data
    latent_codes = encoder.predict(data, verbose=0)

    # Fit a Gaussian distribution to the latent codes
    latent_mean = np.mean(latent_codes, axis=0)
    latent_cov = np.cov(latent_codes, rowvar=False)

    # Ensure covariance matrix is positive definite
    if len(latent_mean) > 1:
        # Add a small value to the diagonal to ensure positive definiteness
        latent_cov += np.eye(latent_cov.shape[0]) * 1e-6

    def generate_samples(n_samples):
        # Sample from the learned latent distribution
        if len(latent_mean) == 1:
            # 1D case
            latent_samples = np.random.normal(latent_mean[0], np.sqrt(latent_cov), (n_samples, 1))
        else:
            # Multi-dimensional case
            try:
                latent_samples = np.random.multivariate_normal(latent_mean, latent_cov, n_samples)
            except:
                # If covariance is not positive definite, use diagonal approximation
                latent_cov_diag = np.diag(np.diag(latent_cov))
                latent_samples = np.random.multivariate_normal(latent_mean, latent_cov_diag, n_samples)

        # Decode to generate new samples
        generated_samples = decoder.predict(latent_samples, verbose=0)
        return generated_samples

    return generate_samples, latent_mean, latent_cov

# Quality measures for generative performance
def is_on_cube_surface(point, tolerance=0.1):
    """
    Check if a point is on the cube surface within tolerance
    Args:
        point: 3D point to check
        tolerance: Tolerance for being on the surface
    Returns:
        Boolean indicating if the point is on the cube surface
    """
    # A point is on the cube surface if at least one coordinate is ±1 within tolerance
    # and the other coordinates are within [-1, 1]
    on_surface = False
    for i in range(3):
        if abs(abs(point[i]) - 1) < tolerance:
            # Check if other coordinates are within bounds
            other_coords = [j for j in range(3) if j != i]
            if all(-1 <= point[j] <= 1 for j in other_coords):
                on_surface = True
                break
    return on_surface

def kl_divergence(p, q):
    """
    Compute KL divergence between two distributions
    Args:
        p: First distribution (reference)
        q: Second distribution (approximation)
    Returns:
        KL divergence value
    """
    # Ensure distributions are normalized
    p = p / np.sum(p)
    q = q / np.sum(q)

    # Avoid zeros for KL calculation
    p = np.clip(p, 1e-10, 1)
    q = np.clip(q, 1e-10, 1)

    return np.sum(p * np.log(p / q))

def evaluate_generative_performance(generate_samples, n_samples=1000):
    """
    Evaluate the quality of generated samples
    Args:
        generate_samples: Function to generate samples
        n_samples: Number of samples to generate
    Returns:
        surface_percentage: Percentage of samples on cube surface
        kl_divergence: KL divergence between original and generated distributions
        samples: Generated samples
    """
    # Generate samples
    samples = generate_samples(n_samples)

    # Calculate percentage of samples on cube surface
    on_surface_count = 0
    for sample in samples:
        if is_on_cube_surface(sample):
            on_surface_count += 1

    surface_percentage = on_surface_count / n_samples * 100

    # Calculate distribution similarity using KL divergence
    # We'll compare the distribution of distances from origin and angles
    original_distances = np.linalg.norm(train_data, axis=1)
    generated_distances = np.linalg.norm(samples, axis=1)

    # Calculate histogram-based KL divergence for distances
    hist_original_dist, bin_edges_dist = np.histogram(original_distances, bins=50, density=True)
    hist_generated_dist, _ = np.histogram(generated_distances, bins=bin_edges_dist, density=True)

    kl_dist = kl_divergence(hist_original_dist, hist_generated_dist)

    # Calculate KL divergence for angular distribution (if 2D or 3D)
    if samples.shape[1] >= 2:
        # For 3D data, we can compute angles
        original_angles = np.arctan2(train_data[:, 1], train_data[:, 0])
        generated_angles = np.arctan2(samples[:, 1], samples[:, 0])

        hist_original_ang, bin_edges_ang = np.histogram(original_angles, bins=50, density=True)
        hist_generated_ang, _ = np.histogram(generated_angles, bins=bin_edges_ang, density=True)

        kl_ang = kl_divergence(hist_original_ang, hist_generated_ang)

        # Combine KL divergences
        total_kl = kl_dist + kl_ang
    else:
        total_kl = kl_dist

    return surface_percentage, total_kl, samples

# Evaluate generative performance for different latent dimensions
print("\nGenerative Performance Evaluation:")
print("Dim\tSurface %\tKL Divergence")
generative_performance = {}

for dim in latent_dims:
    encoder = autoencoders[dim][1]
    decoder = autoencoders[dim][2]

    generate_samples, latent_mean, latent_cov = create_generative_system(encoder, decoder, train_data)
    surface_percentage, kl_value, samples = evaluate_generative_performance(generate_samples)

    generative_performance[dim] = {
        'surface_percentage': surface_percentage,
        'kl_divergence': kl_value  # Use the new variable name
    }
    print(f"{dim}\t{surface_percentage:.2f}%\t\t{kl_value:.4f}")

# Visualize generated samples for the best model
best_dim = min(generative_performance, key=lambda k: generative_performance[k]['kl_divergence'])
encoder = autoencoders[best_dim][1]
decoder = autoencoders[best_dim][2]
generate_samples, _, _ = create_generative_system(encoder, decoder, train_data)
_, _, generated_samples = evaluate_generative_performance(generate_samples, 1000)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(generated_samples[:, 0], generated_samples[:, 1], generated_samples[:, 2], alpha=0.5)
ax.set_title(f'Generated Samples (Best Model: Dim={best_dim})')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1.5, 1.5)
plt.savefig('generated_samples.png')
plt.close()

###############################################################################
# 7. Improved Mutual Information Estimation using k-NN method
###############################################################################
print("\n=== 7. Improved Mutual Information Estimation ===")

def knn_mi_estimation(x, y, k=5):
    """
    Estimate mutual information between x and y using k-NN method
    Based on Kraskov et al. (2004) method
    Args:
        x: First variable (n_samples, n_features_x)
        y: Second variable (n_samples, n_features_y)
        k: Number of nearest neighbors to use
    Returns:
        mi: Estimated mutual information in bits
    """
    n_samples = x.shape[0]

    # Combine x and y
    xy = np.hstack((x, y))

    # Find k-nearest neighbors in the joint space
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(xy)
    distances, _ = nbrs.kneighbors(xy)

    # The k+1-th neighbor is the farthest (we exclude the point itself)
    epsilon = distances[:, -1]

    # Count neighbors in x and y spaces within epsilon
    n_x = np.zeros(n_samples)
    n_y = np.zeros(n_samples)

    for i in range(n_samples):
        # Count points in x-space within epsilon
        n_x[i] = np.sum(np.linalg.norm(x - x[i], axis=1) < epsilon[i])

        # Count points in y-space within epsilon
        n_y[i] = np.sum(np.linalg.norm(y - y[i], axis=1) < epsilon[i])

    # Estimate mutual information using Kraskov's formula
    psi_n = psi(n_samples)
    psi_k = psi(k)

    mi = psi_n + psi_k - np.mean(psi(n_x + 1) + psi(n_y + 1))

    # Convert from nats to bits
    mi_bits = mi / np.log(2)

    return max(0, mi_bits)

def psi(n):
    """Digamma function approximation for large n"""
    return np.log(n) - 1/(2*n) - 1/(12*n**2) + 1/(120*n**4)

# Calculate mutual information between input and latent representations
mi_results = {}
for dim in latent_dims:
    encoder = autoencoders[dim][1]
    latent_codes = encoder.predict(test_data, verbose=0)

    # For each dimension, calculate MI with input
    total_mi = 0
    for i in range(latent_codes.shape[1]):  # For each latent dimension
        latent_dim_i = latent_codes[:, i].reshape(-1, 1)
        mi = knn_mi_estimation(test_data, latent_dim_i, k=5)
        total_mi += mi

    mi_results[dim] = total_mi
    print(f"Latent Dimension: {dim}, Mutual Information: {total_mi:.4f} bits")

# Plot information vs reconstruction error
mi_values = [mi_results[dim] for dim in latent_dims]
mse_values = [histories[dim]['val_loss'][-1] for dim in latent_dims]

plt.figure(figsize=(10, 6))
plt.plot(mi_values, mse_values, 'o-')
for i, dim in enumerate(latent_dims):
    plt.annotate(f'dim={dim}', (mi_values[i], mse_values[i]), xytext=(5, 5), textcoords='offset points')
plt.xlabel('Mutual Information (bits)')
plt.ylabel('Reconstruction Error (MSE)')
plt.title('Information-Reconstruction Trade-off')
plt.grid(True)
plt.savefig('information_reconstruction_tradeoff.png')
plt.close()

###############################################################################
# 8. Compare with VAE approach (theoretical comparison)
###############################################################################
print("\n=== 8. Comparison with VAE Approach ===")
print("Our noise-based approach controls information flow by adding Gaussian noise")
print("to the latent representation, which is similar to the VAE's approach of")
print("modeling the latent distribution as Gaussian.")
print()
print("Key differences:")
print("1. Our approach explicitly controls SNR, while VAE uses KL divergence")
print("2. VAE learns both mean and variance of the latent distribution")
print("3. Our MMD regularization encourages Gaussian latent distribution like VAE")
print("4. VAE's ELBO objective combines reconstruction and distribution matching")
print()
print("Both approaches aim to create a well-structured latent space suitable")
print("for generation, but VAE provides a more principled probabilistic framework.")

###############################################################################
# 9. Summary of Results
###############################################################################
print("\n" + "="*50)
print("SUMMARY OF RESULTS")
print("="*50)

print("\n1. Autoencoder Performance by Latent Dimension:")
print("Dim\tFinal MSE\tMutual Info (bits)")
for dim in latent_dims:
    final_mse = histories[dim]['val_loss'][-1]
    mi = mi_results[dim]
    print(f"{dim}\t{final_mse:.6f}\t{mi:.4f}")

print("\n2. Information Rate at Different SNRs (for dim=2):")
encoder = autoencoders[2][1]
for snr_db in snr_values:
    _, info_rate, _, _ = add_latent_noise(encoder, test_data, snr_db)
    print(f"SNR: {snr_db} dB -> Info Rate: {info_rate:.4f} bits/sample")

print("\n3. Generative Performance:")
print("Dim\tSurface %\tKL Divergence")
for dim in latent_dims:
    perf = generative_performance[dim]
    print(f"{dim}\t{perf['surface_percentage']:.2f}%\t\t{perf['kl_divergence']:.4f}")

print("\n4. Key Observations:")
print("- Higher latent dimensions allow more information preservation but may overfit")
print("- Very low SNRs (<0 dB) destroy most information and yield poor reconstructions")
print("- The optimal latent dimension for generation appears to be around 2-4")
print("- The autoencoder learns to capture the cube surface structure reasonably well")
print("- Information rate increases with SNR as expected from theory")
print("- MMD regularization helps structure the latent space for better generation")

# Save results to file
with open('results_summary.txt', 'w') as f:
    f.write("RESULTS SUMMARY\n")
    f.write("===============\n\n")

    f.write("Autoencoder Performance by Latent Dimension:\n")
    f.write("Dim\tFinal MSE\tMutual Info (bits)\n")
    for dim in latent_dims:
        final_mse = histories[dim]['val_loss'][-1]
        mi = mi_results[dim]
        f.write(f"{dim}\t{final_mse:.6f}\t{mi:.4f}\n")

    f.write("\nInformation Rate at Different SNRs (for dim=2):\n")
    for snr_db in snr_values:
        _, info_rate, _, _ = add_latent_noise(encoder, test_data, snr_db)
        f.write(f"SNR: {snr_db} dB -> Info Rate: {info_rate:.4f} bits/sample\n")

    f.write("\nGenerative Performance:\n")
    f.write("Dim\tSurface %\tKL Divergence\n")
    for dim in latent_dims:
        perf = generative_performance[dim]
        f.write(f"{dim}\t{perf['surface_percentage']:.2f}%\t\t{perf['kl_divergence']:.4f}\n")

print("\nResults saved to 'results_summary.txt'")

=== 1. Generating 3D Cube Surface Data ===
Generated 6000 samples on cube surface
Data range: x[-1.00, 1.00], y[-1.00, 1.00], z[-1.00, 1.00]

=== 2. Building Autoencoder with MSE Objective and MMD Regularization ===
Training autoencoder with latent dimension 1


# Autoencoder Assignment 1: Information-Theoretic Autoencoder

This repository contains the implementation and analysis of an autoencoder applied to data uniformly distributed on the surface of a 3D cube. The project explores the trade-off between reconstruction quality and information preservation in the latent space, incorporating Maximum Mean Discrepancy (MMD) regularization and analyzing mutual information using a k-NN based method.

## Project Structure

The code is structured to follow the assignment requirements step-by-step:

1.  **Data Generation**: Creates 3D data points uniformly distributed on the surface of a cube.
2.  **Autoencoder Implementation**: Builds an autoencoder model with a configurable latent dimension and incorporates MMD regularization to shape the latent distribution.
3.  **Probabilistic Interpretation**: Discusses the probabilistic interpretation of the Mean Squared Error (MSE) objective function.
4.  **Latent Space Control and Noise**: Implements a method to add Gaussian noise to the latent representation based on Signal-to-Noise Ratio (SNR) and calculates the theoretical information rate.
5.  **Reconstruction Analysis**: Evaluates the autoencoder's reconstruction performance at various latent dimensions and SNRs.
6.  **Generative System**: Creates a generative model by fitting a Gaussian distribution to the learned latent space and evaluates the quality of generated samples.
7.  **Mutual Information Estimation**: Uses a k-NN based method to estimate the mutual information between the input data and the latent representation.
8.  **VAE Comparison**: Provides a theoretical comparison of the implemented approach with Variational Autoencoders (VAEs).
9.  **Results Summary**: Summarizes the key findings and observations from the experiments.

## Results Summary

Based on the experiments conducted with different latent dimensions and SNRs, the following key results and observations were made:

| Latent Dimension | Final MSE  | Mutual Info (bits) | Surface % | KL Divergence |
|------------------|------------|--------------------|-----------|---------------|
| 1                | 0.099204   | 2.2934             | 10.60%    | 2.8686        |
| 2                | 0.000817   | 4.6710             | 98.50%    | 0.1485        |
| 4                | 0.000274   | 7.5805             | 20.90%    | 0.1698        |
| 8                | 0.000116   | 16.2127            | 12.50%    | 0.2286        |
| 16               | 0.000103   | 32.9915            | 21.70%    | 0.1406        |

Information Rate at Different SNRs (for dim=2):

| SNR (dB) | Info Rate (bits/sample) |
|----------|-------------------------|
| -10      | 0.0688                  |
| -5       | 0.1982                  |
| 0        | 0.5000                  |
| 5        | 1.0287                  |
| 10       | 1.7297                  |
| 20       | 3.3291                  |
| 30       | 4.9836                  |

Key Observations:

*   Higher latent dimensions generally lead to lower reconstruction error but may capture more noise or irrelevant information, potentially hindering generative performance.
*   Very low SNRs significantly degrade reconstruction quality due to excessive noise in the latent space.
*   The optimal latent dimension for generating samples that resemble the original cube surface appears to be around 2-4, balancing reconstruction quality and latent space structure.
*   The autoencoder effectively learns to represent the cube surface structure in the latent space.
*   The information rate between the input and noisy latent representation increases with SNR, consistent with information theory.
*   MMD regularization helps to shape the latent distribution towards a standard Gaussian, which is beneficial for the generative process.

## Files

*   `autoencoder_assignment.ipynb`: The main notebook containing the code implementation and analysis.
*   `cube_surface_data.png`: Visualization of the generated 3D cube surface data.
*   `training_history_dim_*.png`: Plots showing the training history (loss curves) for autoencoders with different latent dimensions.
*   `reconstruction_comparison.png`: Visualization comparing original test data, reconstructed data, and the distribution of reconstruction errors for a specific latent dimension and SNR.
*   `generated_samples.png`: Visualization of samples generated by the best-performing generative model.
*   `information_reconstruction_tradeoff.png`: Plot showing the relationship between mutual information and reconstruction error for different latent dimensions.
*   `results_summary.txt`: A text file containing the summarized results.

## How to Run

1.  Open the `autoencoder_assignment.ipynb` notebook in Google Colab or a similar environment.
2.  Run all the cells sequentially.
3.  The code will generate the data, train autoencoders with different latent dimensions, perform analysis, and save plots and a results summary file.

## Requirements

The code requires the following libraries:

*   `numpy`
*   `tensorflow`
*   `matplotlib`
*   `sklearn`
*   `scipy`
*   `seaborn`

These can be installed using pip:

# Installation Instructions

To run the code in this notebook, you need to install the following libraries:

*   `numpy`
*   `tensorflow`
*   `matplotlib`
*   `sklearn`
*   `scipy`
*   `seaborn`

These can be installed using pip: